In [1]:
# Análise de Elegibilidade de Crédito

## Este notebook implementa um modelo de classificação para prever a elegibilidade de crédito de solicitantes com base em suas características financeiras. O objetivo é classificar as solicitações em três categorias:

### 1. **Não Elegível** - Solicitações que foram recusadas
### 2. **Elegível com Análise** - Solicitações que necessitam de análise mais detalhada
### 3. **Elegível** - Solicitações aprovadas imediatamente

## Objetivos
### - Explorar e visualizar os dados
### - Criar features derivadas relevantes
### - Treinar modelos KNN e K-Means
### - Comparar o desempenho dos modelos
### - Salvar o melhor modelo para uso em produção

In [2]:
## 1. Configuração Inicial e Importações

### Vamos configurar o ambiente de trabalho, importar as bibliotecas necessárias e definir os parâmetros globais para a análise.

In [ ]:
# Importações
import sys
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import logging
import warnings
from datetime import datetime
from pathlib import Path

# Adicionar diretório raiz ao sys.path
sys.path.append('..')

# Configurar logging
from src.utils.logging_utils import setup_logger
logger = setup_logger("notebook", level=logging.INFO)
logger.info("Iniciando análise de elegibilidade de crédito")

# Suprimir warnings
warnings.filterwarnings('ignore')

# Configuração para visualizações
%matplotlib inline
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 8)

# Importar módulos do projeto
from src import data_processing, model_training, model_evaluation, model_utils

In [ ]:
# Configuração global para análise
RUN_EXPLORATORY_ANALYSIS = True    # Executar análise exploratória dos dados
RUN_FEATURE_ENGINEERING = True     # Criar features derivadas
RUN_KNN_MODEL = True               # Treinar e avaliar modelo KNN
RUN_KMEANS_MODEL = True            # Treinar e avaliar modelo K-Means
RUN_MODEL_COMPARISON = True        # Comparar os modelos
RUN_SAVE_FINAL_MODEL = True        # Salvar o modelo final

# Configurações de comportamento
USE_FEATURE_SCALING = True         # Normalizar as features
USE_ABSOLUTE_PATHS = True          # Usar caminhos absolutos para salvar arquivos
USE_DATA_SPLITTING = True          # Dividir dados em treino/teste
USE_CROSS_VALIDATION = False       # Usar validação cruzada

# Configurações de visualizações
SAVE_VISUALIZATIONS = True         # Salvar as visualizações geradas
VISUALIZATIONS_DIRECTORY = '../visualizacoes'  # Diretório para salvar visualizações

# Parâmetros do modelo
KNN_K_VALUES = [5, 7, 11, 15, 21, 31, 41, 51]  # Valores de K para testar
KMEANS_CLUSTERS = 3                # Número de clusters para K-Means
TEST_SIZE = 0.2                    # Proporção dos dados para teste
RANDOM_STATE = 42                  # Semente para reprodutibilidade

# Features selecionadas para o modelo
SELECTED_FEATURES = [
    'salario_anual', 
    'total_dividas', 
    'historico_pagamento', 
    'razao_endividamento', 
    'capacidade_pagamento'
]

# Verificar/criar diretório para visualizações
if SAVE_VISUALIZATIONS:
    from src.model_evaluation import criar_diretorio_visualizacoes
    viz_dir = criar_diretorio_visualizacoes(VISUALIZATIONS_DIRECTORY, USE_ABSOLUTE_PATHS)
    logger.info(f"Visualizações serão salvas em: {viz_dir}")

In [ ]:
# Configurar TensorBoard (opcional)
USE_TENSORBOARD = True

if USE_TENSORBOARD:
    try:
        from src.utils.tensorboard_utils import MLSummaryWriter
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        tensorboard = MLSummaryWriter(f"notebook_run_{timestamp}")
        logger.info(f"TensorBoard configurado em {tensorboard.log_dir}")
        print(f"TensorBoard configurado. Para visualizar, execute: tensorboard --logdir={tensorboard.log_dir}")
    except Exception as e:
        logger.warning(f"Erro ao configurar TensorBoard: {e}")
        USE_TENSORBOARD = False
        print("TensorBoard não pôde ser configurado. A análise continuará sem ele.")

In [6]:
## 2. Carregamento e Visualização Inicial dos Dados

### Vamos carregar os dados do arquivo CSV e fazer uma inspeção inicial para entender sua estrutura.

In [ ]:
# Caminho para o arquivo CSV
caminho_arquivo = '../data/elegibilidade_credito.csv'

# Carregar dados
try:
    df = data_processing.carregar_dados(caminho_arquivo)
    logger.info(f"Dados carregados: {df.shape[0]} linhas, {df.shape[1]} colunas")
    
    # Exibir primeiras linhas
    print("Primeiras linhas dos dados:")
    display(df.head())
    
    # Informações do DataFrame
    print("\nInformações do DataFrame:")
    df.info()
    
    # Estatísticas descritivas
    print("\nEstatísticas descritivas:")
    display(df.describe())
    
except Exception as e:
    logger.error(f"Erro ao carregar dados: {str(e)}")
    raise

In [8]:
## 3. Engenharia de Features

### Vamos processar os dados brutos e criar features derivadas que podem ser úteis para o modelo:

### 1. Converter o histórico de pagamento para um formato numérico adequado
### 2. Criar a razão de endividamento (dívidas/salário)
### 3. Calcular a capacidade de pagamento ((salário-dívidas)/crédito)

In [ ]:
if RUN_FEATURE_ENGINEERING:
    try:
        logger.info("Iniciando processamento e engenharia de features")
        
        # Processar histórico de pagamento
        df = data_processing.processar_historico_pagamento(df)
        
        # Criar features derivadas
        df = data_processing.criar_features_derivadas(df)
        
        # Verificar as novas colunas
        print("DataFrame após processamento:")
        display(df.head())
        
        # Estatísticas das novas features
        print("\nEstatísticas das features derivadas:")
        display(df[['razao_endividamento', 'capacidade_pagamento']].describe())
        
        logger.info("Engenharia de features concluída com sucesso")
    except Exception as e:
        logger.error(f"Erro na engenharia de features: {str(e)}")
        raise

In [10]:
## 4. Análise Exploratória dos Dados

### Vamos analisar os dados para entender melhor a distribuição das variáveis e a relação entre elas. Isso ajudará a identificar padrões e características importantes para o modelo.

In [ ]:
if RUN_EXPLORATORY_ANALYSIS:
    try:
        logger.info("Iniciando análise exploratória dos dados")
        
        # 1. Distribuição das classes de elegibilidade
        print("Distribuição das classes de elegibilidade:")
        class_counts = df['elegibilidade'].value_counts()
        display(class_counts)
        
        model_evaluation.plotar_distribuicao_classes(
            df, salvar=SAVE_VISUALIZATIONS, 
            diretorio=VISUALIZATIONS_DIRECTORY,
            usar_caminho_absoluto=USE_ABSOLUTE_PATHS
        )
        
        # 2. Boxplots das features por categoria
        print("\nDistribuição das features por categoria de elegibilidade:")
        model_evaluation.plotar_boxplots_por_categoria(
            df, SELECTED_FEATURES, 
            salvar=SAVE_VISUALIZATIONS, 
            diretorio=VISUALIZATIONS_DIRECTORY,
            usar_caminho_absoluto=USE_ABSOLUTE_PATHS
        )
        
        logger.info("Visualizações de distribuição geradas com sucesso")
    except Exception as e:
        logger.error(f"Erro na análise exploratória: {str(e)}")

In [ ]:
if RUN_EXPLORATORY_ANALYSIS:
    try:
        # 3. Matriz de correlação entre as variáveis
        print("\nMatriz de correlação entre as variáveis:")
        features_para_correlacao = SELECTED_FEATURES + ['elegibilidade']
        
        model_evaluation.plotar_matriz_correlacao(
            df, features=features_para_correlacao,
            salvar=SAVE_VISUALIZATIONS, 
            diretorio=VISUALIZATIONS_DIRECTORY,
            usar_caminho_absoluto=USE_ABSOLUTE_PATHS
        )
        
        # 4. Médias das features por categoria
        print("\nMédia das features por categoria de elegibilidade:")
        medias_por_categoria = model_evaluation.plotar_medias_por_categoria(
            df, SELECTED_FEATURES,
            salvar=SAVE_VISUALIZATIONS, 
            diretorio=VISUALIZATIONS_DIRECTORY,
            usar_caminho_absoluto=USE_ABSOLUTE_PATHS
        )
        display(medias_por_categoria)
        
        # 5. Scatter plot de features derivadas
        print("\nRelação entre razão de endividamento e capacidade de pagamento:")
        model_evaluation.plotar_scatter_derivadas(
            df, salvar=SAVE_VISUALIZATIONS, 
            diretorio=VISUALIZATIONS_DIRECTORY,
            usar_caminho_absoluto=USE_ABSOLUTE_PATHS
        )
        
        logger.info("Visualizações de relações geradas com sucesso")
    except Exception as e:
        logger.error(f"Erro na análise de correlação: {str(e)}")

In [13]:
## 5. Preparação dos Dados para Modelagem

### Vamos preparar os dados para treinar os modelos:
### 1. Dividir em conjuntos de treino e teste
### 2. Normalizar as features para facilitar o treinamento
### 3. Aplicar PCA para visualização e análise dos dados em 2D

In [ ]:
try:
    logger.info("Preparando dados para modelagem")
    
    if USE_DATA_SPLITTING:
        # Dividir em treino/teste e normalizar
        X_train_norm, X_test_norm, y_train, y_test, scaler = data_processing.preparar_dados(
            df, SELECTED_FEATURES, 
            test_size=TEST_SIZE, 
            random_state=RANDOM_STATE, 
            normalizar=USE_FEATURE_SCALING
        )
        
        print(f"Divisão dos dados:")
        print(f"  - Treino: {X_train_norm.shape[0]} amostras ({(1-TEST_SIZE)*100:.0f}%)")
        print(f"  - Teste: {X_test_norm.shape[0]} amostras ({TEST_SIZE*100:.0f}%)")
        
        # Se estamos normalizando, mostrar as médias e desvios padrão
        if USE_FEATURE_SCALING and scaler is not None:
            print("\nParâmetros de normalização (StandardScaler):")
            for i, feature in enumerate(SELECTED_FEATURES):
                print(f"  - {feature}:")
                print(f"      - Média: {scaler.mean_[i]:.6f}")
                print(f"      - Desvio padrão: {scaler.scale_[i]:.6f}")
    else:
        # Usar todos os dados (sem divisão)
        print("Usando todos os dados para treinamento (sem divisão treino/teste)")
        X = df[SELECTED_FEATURES].values
        y = df['elegibilidade'].values
        
        if USE_FEATURE_SCALING:
            X_norm, scaler = data_processing.normalizar_dados(X)
            X_train_norm, X_test_norm = X_norm, X_norm
        else:
            X_train_norm, X_test_norm = X, X
            scaler = None
            
        y_train, y_test = y, y
        
        print(f"Total de amostras: {X.shape[0]}")
    
    # Visualização PCA
    if RUN_EXPLORATORY_ANALYSIS:
        print("\nAplicando PCA para visualização em 2D:")
        pca = model_evaluation.plotar_pca(
            X_train_norm, y_train, 
            salvar=SAVE_VISUALIZATIONS, 
            diretorio=VISUALIZATIONS_DIRECTORY,
            usar_caminho_absoluto=USE_ABSOLUTE_PATHS
        )
    
    logger.info("Preparação dos dados concluída com sucesso")
    
except Exception as e:
    logger.error(f"Erro na preparação dos dados: {str(e)}")
    raise

In [15]:
## 6. Modelo KNN (K-Nearest Neighbors)

### Vamos treinar um modelo KNN (K-Vizinhos Mais Próximos) para classificar a elegibilidade de crédito.
### Testaremos diferentes valores de K para encontrar o melhor.

In [ ]:
if RUN_KNN_MODEL:
    try:
        print("\n" + "="*50)
        print("TREINAMENTO E AVALIAÇÃO DO MODELO KNN")
        print("="*50)
        logger.info("Iniciando treinamento do modelo KNN")
        
        if USE_CROSS_VALIDATION:
            # Validação cruzada (mais robusta, mas mais lenta)
            print("\nRealizando validação cruzada para diferentes valores de K:")
            resultados_knn, melhor_k = model_training.avaliar_knn_cross_validation(
                X_train_norm, y_train, KNN_K_VALUES
            )
            
            print("\nResultados da validação cruzada:")
            for k, acuracia in resultados_knn.items():
                print(f"  K = {k}: {acuracia:.4f}")
        else:
            # Avaliação direta no conjunto de teste
            print("\nAvaliando KNN com diferentes valores de K:")
            resultados_knn = {}
            
            for k in KNN_K_VALUES:
                # Treinar modelo
                modelo = model_training.treinar_knn(X_train_norm, y_train, k)
                
                # Avaliar modelo
                acuracia, _ = model_evaluation.avaliar_modelo(modelo, X_test_norm, y_test)
                resultados_knn[k] = acuracia
                
                print(f"  K = {k}: {acuracia:.4f}")
        
        # Visualizar resultados para diferentes valores de K
        acuracias = [resultados_knn[k] for k in KNN_K_VALUES]
        melhor_k, melhor_acuracia = model_evaluation.plotar_acuracia_diferentes_k(
            KNN_K_VALUES, acuracias,
            salvar=SAVE_VISUALIZATIONS, 
            diretorio=VISUALIZATIONS_DIRECTORY,
            usar_caminho_absoluto=USE_ABSOLUTE_PATHS
        )
        
        print(f"\nMelhor valor de K: {melhor_k} com acurácia de {melhor_acuracia:.4f}")
        
        # Registrar no TensorBoard
        if USE_TENSORBOARD:
            tensorboard.log_knn_accuracy_vs_k(KNN_K_VALUES, acuracias)
        
        logger.info(f"Melhor valor de K para KNN: {melhor_k}")
        
    except Exception as e:
        logger.error(f"Erro no treinamento do KNN: {str(e)}")
        if 'resultados_knn' not in locals():
            resultados_knn = {}
            melhor_k = 21  # valor padrão em caso de erro

In [ ]:
if RUN_KNN_MODEL:
    try:
        # Treinar o modelo final com o melhor valor de K
        print(f"\nTreinando o modelo KNN final com K = {melhor_k}...")
        modelo_knn = model_training.treinar_knn(X_train_norm, y_train, melhor_k)
        
        # Avaliar o modelo
        accuracy_knn, y_pred_knn = model_evaluation.avaliar_modelo(modelo_knn, X_test_norm, y_test)
        print(f"Acurácia do modelo KNN final: {accuracy_knn:.4f}")
        
        # Matriz de confusão
        print("\nMatriz de confusão:")
        model_evaluation.plotar_matriz_confusao(
            y_test, y_pred_knn,
            salvar=SAVE_VISUALIZATIONS, 
            diretorio=VISUALIZATIONS_DIRECTORY,
            usar_caminho_absoluto=USE_ABSOLUTE_PATHS
        )
        
        # Visualizar fronteira de decisão (usando as duas features mais informativas)
        print("\nFronteira de decisão (usando razão de endividamento e capacidade de pagamento):")
        model_evaluation.plotar_fronteira_decisao(
            modelo_knn, X_test_norm, y_test, 
            SELECTED_FEATURES, [3, 4],  # índices das features razao_endividamento e capacidade_pagamento
            salvar=SAVE_VISUALIZATIONS, 
            diretorio=VISUALIZATIONS_DIRECTORY,
            usar_caminho_absoluto=USE_ABSOLUTE_PATHS
        )
        
        # Registrar no TensorBoard
        if USE_TENSORBOARD:
            from sklearn.metrics import confusion_matrix
            cm = confusion_matrix(y_test, y_pred_knn)
            tensorboard.log_confusion_matrix(
                cm, ['Não Elegível', 'Elegível c/ Análise', 'Elegível']
            )
            tensorboard.log_metrics({'KNN_accuracy': accuracy_knn})
        
        logger.info(f"Modelo KNN treinado e avaliado com sucesso. Acurácia: {accuracy_knn:.4f}")
        
    except Exception as e:
        logger.error(f"Erro na avaliação do KNN: {str(e)}")
        if 'accuracy_knn' not in locals():
            accuracy_knn = 0
            y_pred_knn = []

In [18]:
## 7. Modelo K-Means

### Vamos treinar um modelo K-Means com 3 clusters, para as três categorias de elegibilidade.
### K-Means é um algoritmo de aprendizado não supervisionado que agrupa os dados em clusters.

In [ ]:
if RUN_KMEANS_MODEL:
    try:
        print("\n" + "="*50)
        print("TREINAMENTO E AVALIAÇÃO DO MODELO K-MEANS")
        print("="*50)
        logger.info(f"Iniciando treinamento do modelo K-Means com {KMEANS_CLUSTERS} clusters")
        
        # Treinar modelo K-Means
        modelo_kmeans = model_training.treinar_kmeans(
            X_train_norm, n_clusters=KMEANS_CLUSTERS, random_state=RANDOM_STATE
        )
        
        # Mapear clusters para classes
        mapeamento_clusters = model_training.mapear_clusters_kmeans(
            modelo_kmeans, X_train_norm, y_train
        )
        
        print("\nMapeamento de clusters para classes:")
        for cluster, classe in mapeamento_clusters.items():
            categorias = {1: 'Não Elegível', 2: 'Elegível c/ Análise', 3: 'Elegível'}
            print(f"  Cluster {cluster} -> Classe {classe} ({categorias[classe]})")
        
        # Fazer previsões
        clusters_teste = modelo_kmeans.predict(X_test_norm)
        y_pred_kmeans = np.array([mapeamento_clusters[c] for c in clusters_teste])
        
        # Calcular acurácia
        accuracy_kmeans = (y_pred_kmeans == y_test.values).mean()
        print(f"\nAcurácia do modelo K-Means: {accuracy_kmeans:.4f}")
        
        # Matriz de confusão
        print("\nMatriz de confusão:")
        model_evaluation.plotar_matriz_confusao(
            y_test, y_pred_kmeans,
            salvar=SAVE_VISUALIZATIONS, 
            diretorio=VISUALIZATIONS_DIRECTORY,
            usar_caminho_absoluto=USE_ABSOLUTE_PATHS
        )
        
        # Registrar no TensorBoard
        if USE_TENSORBOARD:
            from sklearn.metrics import confusion_matrix
            cm = confusion_matrix(y_test, y_pred_kmeans)
            tensorboard.log_confusion_matrix(cm, ['Não Elegível', 'Elegível c/ Análise', 'Elegível'], step=1)
            tensorboard.log_metrics({'KMeans_accuracy': accuracy_kmeans})
        
        logger.info(f"Modelo K-Means treinado e avaliado com sucesso. Acurácia: {accuracy_kmeans:.4f}")
        
    except Exception as e:
        logger.error(f"Erro no modelo K-Means: {str(e)}")
        if 'accuracy_kmeans' not in locals():
            accuracy_kmeans = 0
            y_pred_kmeans = []

In [20]:
## 8. Comparação dos Modelos

### Vamos comparar o desempenho dos modelos KNN e K-Means e selecionar o melhor.

In [ ]:
if RUN_MODEL_COMPARISON and RUN_KNN_MODEL and RUN_KMEANS_MODEL:
    try:
        print("\n" + "="*50)
        print("COMPARAÇÃO DOS MODELOS")
        print("="*50)
        logger.info("Iniciando comparação dos modelos")
        
        # Comparar visualmente
        model_evaluation.comparar_modelos(
            ['KNN', 'K-Means'], 
            [accuracy_knn, accuracy_kmeans],
            salvar=SAVE_VISUALIZATIONS, 
            diretorio=VISUALIZATIONS_DIRECTORY,
            usar_caminho_absoluto=USE_ABSOLUTE_PATHS
        )
        
        # Selecionar o melhor modelo
        if accuracy_knn > accuracy_kmeans:
            best_model = 'KNN'
            modelo_final = modelo_knn
            best_accuracy = accuracy_knn
        else:
            best_model = 'K-Means'
            modelo_final = modelo_kmeans
            best_accuracy = accuracy_kmeans
        
        print(f"\nComparação de acurácia:")
        print(f"  - KNN (k={melhor_k}): {accuracy_knn:.4f}")
        print(f"  - K-Means (clusters={KMEANS_CLUSTERS}): {accuracy_kmeans:.4f}")
        print(f"\nMelhor modelo: {best_model} com acurácia={best_accuracy:.4f}")
        
        logger.info(f"Comparação concluída. Melhor modelo: {best_model} com acurácia {best_accuracy:.4f}")
        
    except Exception as e:
        logger.error(f"Erro na comparação dos modelos: {str(e)}")
        if 'best_model' not in locals():
            if 'accuracy_knn' in locals() and 'accuracy_kmeans' in locals():
                best_model = 'KNN' if accuracy_knn > accuracy_kmeans else 'K-Means'
                best_accuracy = max(accuracy_knn, accuracy_kmeans)
                modelo_final = modelo_knn if best_model == 'KNN' else modelo_kmeans
            elif 'accuracy_knn' in locals():
                best_model = 'KNN'
                best_accuracy = accuracy_knn
                modelo_final = modelo_knn
            elif 'accuracy_kmeans' in locals():
                best_model = 'K-Means'
                best_accuracy = accuracy_kmeans
                modelo_final = modelo_kmeans

In [22]:
## 9. Salvar o Modelo Final

### Vamos salvar o melhor modelo e o scaler para uso em produção e criar documentação detalhada.

In [ ]:
if RUN_SAVE_FINAL_MODEL:
    try:
        print("\n" + "="*50)
        print("SALVANDO O MODELO FINAL")
        print("="*50)
        
        # Verificar qual modelo salvar
        if 'best_model' not in locals():
            if RUN_KNN_MODEL:
                best_model = 'KNN'
                modelo_final = modelo_knn
                best_accuracy = accuracy_knn
            elif RUN_KMEANS_MODEL:
                best_model = 'K-Means'
                modelo_final = modelo_kmeans
                best_accuracy = accuracy_kmeans
            else:
                raise ValueError("Nenhum modelo foi treinado para ser salvo.")
        
        print(f"Salvando o modelo {best_model} com acurácia de {best_accuracy:.4f}")
        
        # Salvar modelo e scaler
        model_training.salvar_modelo(
            modelo_final, scaler,
            caminho_modelo='../models/model.joblib',
            caminho_scaler='../models/scaler.joblib'
        )
        
        # Criar documentação
        documentacao = model_utils.criar_documentacao(
            modelo_final, scaler, SELECTED_FEATURES, best_accuracy, 
            caminho_saida='../models/model_documentation.md'
        )
        
        print(f"Modelo salvo com sucesso em ../models/")
        print(f"Documentação criada em ../models/model_documentation.md")
        
        logger.info(f"Modelo {best_model} salvo com sucesso")
        
    except Exception as e:
        logger.error(f"Erro ao salvar modelo: {str(e)}")

In [24]:
## 10. Teste do Modelo com Exemplos

### Vamos testar o modelo com alguns exemplos para ver como ele se comporta na prática.

In [ ]:
if 'modelo_final' in locals() and 'scaler' in locals():
    try:
        print("\n" + "="*50)
        print("TESTANDO O MODELO COM EXEMPLOS")
        print("="*50)
        
        # Testar com exemplos pré-definidos
        resultados_exemplos = model_evaluation.testar_exemplos(
            modelo_final, scaler, SELECTED_FEATURES
        )
        
        logger.info("Testes com exemplos concluídos com sucesso")
        
    except Exception as e:
        logger.error(f"Erro nos testes com exemplos: {str(e)}")

In [26]:
## 11. Conclusão

### Resumo dos principais resultados e insights obtidos na análise.

In [ ]:
print("\n" + "="*50)
print("CONCLUSÃO DA ANÁLISE")
print("="*50)

print("Neste notebook, implementamos e avaliamos modelos de classificação para prever a elegibilidade de crédito.")
print("\nPrincipais conclusões:")

if RUN_EXPLORATORY_ANALYSIS:
    print("\n1. Insights da análise exploratória:")
    print("   - As features mais importantes são razão de endividamento e capacidade de pagamento")
    print("   - Clientes não elegíveis tendem a ter alta razão de endividamento (média de ~98%)")
    print("   - Clientes elegíveis tendem a ter melhor histórico de pagamento")

if RUN_KNN_MODEL:
    print(f"\n2. Modelo KNN:")
    print(f"   - Melhor valor de K: {melhor_k}")
    print(f"   - Acurácia: {accuracy_knn:.4f} ({accuracy_knn*100:.1f}%)")

if RUN_KMEANS_MODEL:
    print(f"\n3. Modelo K-Means:")
    print(f"   - Número de clusters: {KMEANS_CLUSTERS}")
    print(f"   - Acurácia: {accuracy_kmeans:.4f} ({accuracy_kmeans*100:.1f}%)")

if RUN_MODEL_COMPARISON and RUN_KNN_MODEL and RUN_KMEANS_MODEL:
    print(f"\n4. Comparação:")
    print(f"   - Melhor modelo: {best_model}")
    print(f"   - Acurácia: {best_accuracy:.4f} ({best_accuracy*100:.1f}%)")

if RUN_SAVE_FINAL_MODEL:
    print("\n5. Entregáveis:")
    print("   - Modelo salvo em '../models/model.joblib'")
    print("   - Scaler salvo em '../models/scaler.joblib'")
    print("   - Documentação em '../models/model_documentation.md'")

print("\nO modelo pode ser usado como suporte à decisão para avaliar a elegibilidade de crédito.")

# Fechar o TensorBoard writer
if USE_TENSORBOARD:
    try:
        tensorboard.close()
        print(f"\nLogs do TensorBoard salvos em: {tensorboard.log_dir}")
    except:
        pass

logger.info("Análise concluída com sucesso!")

In [ ]:
# Exibir informações para execução em produção
print("\n" + "="*50)
print("PRÓXIMOS PASSOS: EXECUÇÃO EM PRODUÇÃO")
print("="*50)

print("Para executar o modelo em produção via linha de comando, use:")

if 'best_model' in locals() and best_model == 'KNN':
    print(f"\npython ../main.py --model knn --k {melhor_k} --no-visualizations")
elif 'best_model' in locals() and best_model == 'K-Means':
    print(f"\npython ../main.py --model kmeans --clusters {KMEANS_CLUSTERS} --no-visualizations")
else:
    print("\npython ../main.py --model both")
    
print("\nPara processar novos dados:")
print("python ../main.py --data [caminho_para_novos_dados.csv] --output [diretorio_saida/]")